<a href="https://colab.research.google.com/github/KyleHung7/programming-language/blob/main/Week_6_Advanced_2024_Q3_Financial_Report_Analysis_with_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **2024 Q3 上市公司財報分析**

#### 1.從google sheet 匯入原始資料

In [1]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [15]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1OlKQx45xvfCgD3TwCSv7yA8cPFv45zhzM8Hr0sUN0_o/edit?usp=sharing')

In [16]:
from gspread_dataframe import set_with_dataframe

# 選擇要更新的工作表（選擇第一個工作表）
worksheet = gsheets.get_worksheet(0)  # 如果是其他工作表，可以用 get_worksheet(index)

In [17]:
# 取得 Google Sheets 內容
data = worksheet.get_all_values()  # 取得所有儲存格資料，回傳為 2D 陣列

# 將資料轉換為 DataFrame
df = pd.DataFrame(data)

In [18]:
df

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,,,,,,,,,,民國113年1-9月,...,,,,,,,,,單位：新臺幣千元,
1,,,,,,,,,,Jan-Sep 2024,...,,,,,,,,,"UNIT：NT$ 1,000",
2,,,營業收入,,,營業利益,,營業外收入及支出合計,,稅後淨利(淨損),...,本期末,每股盈餘(NT$),,每 股,淨值佔,流動,速動,稅前淨利(淨損),,
3,公司名稱,,Operating Revenue,,,Net Operating Income (Loss),,Total non-operating income and expenses,,Profit (Loss) after Tax,...,股本,Earnings Per Share,,淨 值,總資產,比率,比率,Profit (Loss) before Tax,,
4,Code&Name,,113年1-9月,112年1-9月,增減,113年1-9月,112年1-9月,113年1-9月,112年1-9月,113年1-9月,...,Share Capital,113年1-9月,112年1-9月,(NT$),(%),,Quick,113年1-9月,112年1-9月,增減
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1126,,,,,,,,,,,...,,,,,,,,,,
1127,,合計,"30,264,849,193","26,963,889,905",12.24,"3,247,242,896","2,350,307,698","386,045,336","420,139,358","2,844,450,578",...,"7,864,057,376",3.67,2.79,37.57,20.52,1.56,0.93,"3,633,288,232","2,770,447,056",31.14
1128,,,,,,,,,,,...,,,,,,,,,,
1129,,,,,,,,,,,...,,,,,,,,,,


#### 2.原始資料清理

In [19]:
import pandas as pd

# 設定需要的欄位名稱（移除 Extra_Column1, Extra_Column2, Extra_Column3）
expected_columns = [
    "Code_Name", "Company", "Category", "Revenue_2024", "Revenue_Growth", "Revenue_2023",
    "Net_Income_2024", "Net_Income_2023", "Non_Operating_Income", "Non_Operating_Expense",
    "Profit_After_Tax_2024", "Profit_After_Tax_2023", "Share_Capital",
    "EPS_2024", "EPS_2023", "Net_Value", "Net_Value_Growth", "Quick_Ratio",
    "Current_Ratio", "Pre_Tax_Profit_2024", "Pre_Tax_Profit_2023", "Pre_Tax_Profit_Growth"
]

# 確保欄位數量匹配
df.columns = expected_columns[:len(df.columns)]

# 轉換 Code_Name 為數字
df["Code_Name"] = pd.to_numeric(df["Code_Name"], errors="coerce")

# 建立 Category 欄位
df["Category"] = pd.NA  # 使用 Pandas NA 類型

# 找出類別行 (Code_Name 在 1-18、20、21-38 之間)
category_rows = df[df["Code_Name"].isin(list(range(1, 19)) + [20] + list(range(21, 39)))]

# 迴圈填充 Category
current_category = None
for index, row in df.iterrows():
    if pd.notna(row["Code_Name"]) and row["Code_Name"] in list(range(1, 19)) + [20] + list(range(21, 39)):
        current_category = row["Company"]
    else:
        df.loc[index, "Category"] = current_category  # 使用 .loc 避免警告

# 過濾掉類別行（確保 copy()，避免 SettingWithCopyWarning）
df = df[~df["Code_Name"].isin(list(range(1, 19)) + [20] + list(range(21, 39)))].copy()

# 轉換 Code_Name 為整數
df = df.dropna(subset=["Code_Name"])
df["Code_Name"] = df["Code_Name"].astype(int)

# 重新索引
df = df.reset_index(drop=True)

# 確保沒有多餘的 Extra_Column
df = df[expected_columns]  # 只保留所需欄位


In [20]:
df

,Code_Name,Company,Category,Revenue_2024,Revenue_Growth,Revenue_2023,Net_Income_2024,Net_Income_2023,Non_Operating_Income,Non_Operating_Expense,...,Share_Capital,EPS_2024,EPS_2023,Net_Value,Net_Value_Growth,Quick_Ratio,Current_Ratio,Pre_Tax_Profit_2024,Pre_Tax_Profit_2023,Pre_Tax_Profit_Growth
0,1101,台泥,水泥類,"80,966,464",30.41,"11,795,129","6,596,656","3,525,863","3,943,167","7,363,974",...,"77,511,817",0.93,0.82,31.94,42.1,1.99,1.29,"15,320,992","10,539,823",45.36
1,1102,亞泥,水泥類,"60,239,086",-7.97,"4,767,095","5,992,893","6,131,943","5,995,078","9,087,900",...,"35,465,629",2.72,2.88,50.12,54.06,1.46,0.8,"10,899,038","11,987,971",-9.08
2,1103,嘉泥,水泥類,"2,115,231",4.63,"-81,015","-126,554","680,460","276,448","517,895",...,"7,902,474",0.79,0.21,32.88,60.31,4.18,1.53,"599,445","149,894",299.91
3,1104,環泥,水泥類,"5,627,754",4.37,"782,566","664,888","576,771","1,379,251","1,105,588",...,"6,866,818",1.61,2.4,33.91,80.09,1.84,0.86,"1,359,337","2,044,139",-33.5
4,1108,幸福,水泥類,"3,689,989",-3.66,"469,973","507,659","-16,691","60,557","362,658",...,"4,047,380",0.9,1.16,12.45,49.9,1.44,0.35,"453,282","568,216",-20.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,9940,信義房屋,其他類,"10,807,808",-3.93,"1,643,251","1,657,086","411,054","172,331","1,542,786",...,"7,368,465",2.09,1.84,18.52,39.39,2.64,0.71,"2,054,305","1,829,417",12.29
1020,9941,裕融,其他類,"30,800,878",1.95,"5,090,668","6,734,102","682,240","291,384","4,234,030",...,"6,482,250",7.4,9.31,58.1,12.79,1,0.84,"5,772,908","7,025,486",-17.83
1021,9942,茂順,其他類,"3,657,072",-20.02,"732,616","1,020,773","41,338","43,441","525,722",...,"831,613",6.32,9.41,47.72,78.4,4.94,2.98,"773,954","1,064,214",-27.27
1022,9944,新麗,其他類,"1,642,565",13.57,"170,249","-37,542","-30,864","453,657","35,906",...,"1,091,071",0.34,1.71,32.97,55.71,2.06,1.62,"139,385","416,115",-66.5


#### 3.回傳google sheet 紀錄資料

In [24]:
from gspread_dataframe import set_with_dataframe

# 選擇要更新的工作表（選擇第一個工作表）
worksheet = gsheets.get_worksheet(1)  # 如果是其他工作表，可以用 get_worksheet(index)

# 將 DataFrame 寫入 Google Sheets
set_with_dataframe(worksheet, df)

#### 4.繪製長條圖分析

In [21]:
import pandas as pd
import plotly.express as px

# 確保 Revenue_2024 是數值型態，去掉千分位逗號
df["Revenue_2024"] = df["Revenue_2024"].astype(str).str.replace(",", "").astype(float)

# 先按 2024 年營收排序，取前 10 名
top10_companies = df.nlargest(10, "Revenue_2024")

# 繪製長條圖
fig = px.bar(top10_companies,
             x="Company",
             y="Revenue_2024",
             title="前 10 大企業 2024 年營收比較",
             labels={"Company": "公司名稱", "Revenue_2024": "營收 (NTD)"},
             color="Company")  # 顯示數值

# 顯示圖表
fig.show()


#### 5.繪製折線圖分析

In [22]:
import plotly.express as px
import pandas as pd

# 取出 EPS 相關數據
df_melted_eps = df.melt(id_vars=["Company"],
                         value_vars=["EPS_2023", "EPS_2024"],
                         var_name="Year",
                         value_name="EPS")

# 取前10大企業
top10_companies = df.groupby("Company")["Revenue_2024"].sum().nlargest(10).index
df_top10_eps = df_melted_eps[df_melted_eps["Company"].isin(top10_companies)]

# 繪製折線圖
fig = px.line(df_top10_eps,
              x="Year",
              y="EPS",
              color="Company",
              markers=True,
              title="前 10 大企業 EPS（每股盈餘）趨勢",
              labels={"EPS": "每股盈餘 (NTD)", "Year": "年度"})

fig.show()


#### 6.機器學習分析

In [45]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import silhouette_score

# 1️⃣ Load & Clean Data
df_cleaned = df.drop(columns=["Code_Name", "Company"])  # 移除非數值欄位

# 使用原始類別數值
category_rows = df[df["Code_Name"].isin(list(range(1, 19)) + [20] + list(range(21, 39)))]
df["Category"] = df["Code_Name"].where(df["Code_Name"].isin(category_rows["Code_Name"]), np.nan)

# One-Hot Encoding 加入分類變數
ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
category_encoded = ohe.fit_transform(df[["Category"]].fillna("Unknown"))
category_df = pd.DataFrame(category_encoded, columns=ohe.get_feature_names_out(["Category"]))

# 刪除舊的 Category 欄位，並合併 One-Hot Encoding 結果
df_cleaned = df_cleaned.drop(columns=["Category"]).reset_index(drop=True)
df_cleaned = pd.concat([df_cleaned, category_df], axis=1)
df_cleaned = df_cleaned.drop(columns=["Category_Unknown"], errors="ignore")
# 去除數值欄位中的千分位符號，轉換為數字
df_cleaned = df_cleaned.replace(",", "", regex=True).apply(pd.to_numeric, errors="coerce")

# 去除 NaN 資料並同步 df
df_cleaned = df_cleaned.dropna()
df = df.loc[df_cleaned.index].reset_index(drop=True)

# 2️⃣ Standardize Data
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_cleaned)

# 3️⃣ PCA Dimensionality Reduction
pca = PCA(n_components=2)
pca_result = pca.fit_transform(df_scaled)
df["PCA1"] = pca_result[:, 0]
df["PCA2"] = pca_result[:, 1]

# 4️⃣ Find Best K for K-Means
inertia = []
sil_scores = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(df_scaled)
    inertia.append(kmeans.inertia_)
    sil_scores.append(silhouette_score(df_scaled, kmeans.labels_))

# Elbow Method Plot
fig_elbow = px.line(
    x=list(K_range),
    y=inertia,
    markers=True,
    title="Elbow Method to Find Best K",
    labels={"x": "Number of Clusters (K)", "y": "Inertia (SSE)"},
)
fig_elbow.show()

# 5️⃣ Apply K-Means Clustering
best_k = 3
kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=10)
df["Cluster"] = kmeans.fit_predict(df_scaled)

# 6️⃣ PCA + Clustering Visualization
fig_cluster = px.scatter(
    df, x="PCA1", y="PCA2", color=df["Cluster"].astype(str),
    title="PCA + K-Means Clustering Result",
    labels={"PCA1": "PCA Component 1", "PCA2": "PCA Component 2", "Cluster": "Cluster Group"},
)
fig_cluster.show()

# 7️⃣ Heatmap of Feature Correlations
corr_matrix = df_cleaned.corr()
fig_heatmap = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns,
    y=corr_matrix.index,
    colorscale="viridis"
))
fig_heatmap.update_layout(title="Feature Correlation Heatmap")
fig_heatmap.show()


這段程式碼的目的是從原始資料進行數據清理、標準化、主成分分析 (PCA) 降維、K-means 聚類分析，並最終展示聚類結果和特徵關聯熱圖。以下是每個步驟的詳細說明：

#### 1️⃣ 載入並清理資料 (Load & Clean Data)
  - **移除非數值欄位**：移除資料中的 `Code_Name` 和 `Company` 欄位，這些欄位並非數值型資料。
  - **建立分類欄位**：使用 `Code_Name` 來創建一個新的 `Category` 欄位。只有當 `Code_Name` 在指定的範圍內時，該行會被標註為有效分類，否則標註為 `NaN`。
  - **One-Hot Encoding**：對 `Category` 欄位進行 One-Hot 編碼，將其轉換為數值型欄位。將編碼結果合併到清理過的資料框中，並刪除不再需要的 `Category_Unknown` 欄位。
  - **數字轉換與清理**：去除數值欄位中的千分位符號，並將它們轉換為數值型。刪除包含 `NaN` 的行，並確保資料與已清理的資料同步。

#### 2️⃣ 標準化資料 (Standardize Data)
  - **標準化**：使用 `StandardScaler` 將資料進行標準化處理，將每個特徵的數值轉換為均值為0，標準差為1的分佈。

#### 3️⃣ PCA 主成分分析 (Dimensionality Reduction using PCA)
  - **PCA 降維**：使用 PCA（主成分分析）將資料降至 2 維，這樣可以將高維度資料投影到一個 2D 空間中，便於可視化和進一步分析。將降維後的結果新增為 `PCA1` 和 `PCA2` 欄位。

#### 4️⃣ 尋找最佳的 K 值 (Find Best K for K-Means)
  - **K-means 聚類**：使用 K-means 聚類算法對資料進行聚類，測試不同數量的群集（`K` 值範圍是 2 到 10）來找到最佳的 K 值。透過計算每個 `K` 的 inertia（群內誤差平方和）和 silhouette score（輪廓係數）來評估每個聚類模型的效果。繪製 `Elbow Method` 圖來幫助決定最佳 K 值。

#### 5️⃣ 應用 K-Means 聚類 (Apply K-Means Clustering)
  - **確定最佳 K 值**：根據 `Elbow Method` 和輪廓係數的結果選擇最佳的 K 值（此處選擇 K=3）。使用 K-means 聚類對資料進行分群，並將結果儲存在 `Cluster` 欄位中。

#### 6️⃣ PCA + 聚類可視化 (PCA + Clustering Visualization)
  - **繪製聚類結果圖**：使用 `plotly.express.scatter` 繪製 2 維 PCA 降維後的資料點，並根據 `Cluster` 欄位為資料點著色，以視覺化每個資料點所屬的群集。

#### 7️⃣ 特徵相關性熱圖 (Feature Correlation Heatmap)
  - **計算相關矩陣**：計算資料中各數值特徵之間的相關性（例如 `Revenue_2024`, `Net_Income_2024` 等）。
  - **繪製熱力圖**：使用 `plotly.graph_objects.Heatmap` 繪製特徵之間的相關性熱力圖，顯示不同特徵之間的相關程度，並以顏色來表示強弱。


In [55]:
import plotly.express as px
# 使用 groupby 來按 Cluster 分組，並列出每個群集中的 Company
for cluster_num in df["Cluster"].unique():
    companies_in_cluster = df[df["Cluster"] == cluster_num]["Company"].tolist()
    print(f"Cluster {cluster_num}:")
    print(companies_in_cluster)
    print("="*50)  # 用於分隔不同群集的輸出

# 繪製以 Cluster 為分類的'Revenue_2024箱型圖
fig_revenue = px.box(
    df,
    x="Cluster",
    y="Revenue_2024",
    title="Revenue_2024 Distribution by Cluster",
    labels={"Cluster": "Cluster Group", "Revenue_2024": "Revenue in 2024"}
)
fig_revenue.show()



Cluster 0:
['台泥', '亞泥', '嘉泥', '環泥', '幸福', '信大', '東泥', '味全', '味王', '大成', '卜蜂', '統一', '愛之味', '福壽', '台榮', '福懋油', '佳格', '聯華', '聯華食', '大統益', '天仁', '黑松', '興泰', '鮮活果汁-KY', '南僑', '臺鹽', '南亞', '三芳', '達新', '大洋', '永裕', '台化', '富林-KY', '炎洲', '遠東新', '南染', '東和', '新紡', '利華', '福懋', '力麗', '佳和', '宏益', '台富', '宜進', '聯發', '強盛', '得力', '偉全', '昶和', '大統新創', '台南', '業旺', '儒鴻', '聚陽', '廣越', '冠星-KY', '宜新實業', '士電', '東元', '正道', '中興電', '亞力', '力山', '華城', '鑽全', '樂事綠能', '亞崴', '勤美', '中宇', '廣隆', '巨庭', '喬福', '錩泰', '伸興', '中砂', '程泰', '直得', '上銀', '大同', '瑞智', '力達-KY', '強信-KY', '時碩工業', '鈞興-KY', '駐龍', '大銀微系統', '台灣精銳', '豐祥-KY', '建德工業', '寶一', '高力', '華電', '聲寶', '華新', '華榮', '大亞', '中電', '宏泰', '三洋電', '大山', '億泰', '榮星', '合機', '燦星網', '禾聯碩', '凱撒衛', '正隆', '永豐餘', '永豐實', '東和鋼鐵', '高興昌', '第一銅', '春源', '春雨', '中鋼構', '豐興', '美亞', '大成鋼', '威致', '盛餘', '彰源', '新光鋼', '允強', '海光', '長榮鋼', '豐達科', '三星', '東明-KY', '世紀鋼', '南港', '台橡', '正新', '建大', '厚生', '南帝', '華豐', '鑫永銓', '東陽', '昭輝', '大億', '堤維西', '耿鼎', '江申', '和大', '巧新', '倉佑', '六暉-KY', '裕隆', '中華', '三陽工業', '裕日車', '劍麟',

這段程式碼的目的是根據 `Cluster` 群集標籤，列出每個群集中的公司，並繪製基於群集的 `Revenue_2024` 箱型圖。以下是每個步驟的詳細說明：

#### 1️⃣ 列出每個群集中的公司 (List Companies in Each Cluster)
  - **群集分組**：使用 `groupby` 來按 `Cluster` 欄位將資料分組，然後列出每個群集中的所有公司。
  - **迴圈輸出**：對每個群集，透過 `df[df["Cluster"] == cluster_num]["Company"].tolist()` 來獲取該群集中的公司名稱，並列印出來。
  - **輸出分隔**：在每次輸出不同群集的公司名稱後，使用 `print("="*50)` 來插入分隔線，使輸出更易讀。

#### 2️⃣ 繪製基於群集的 `Revenue_2024` 箱型圖 (Box Plot of Revenue_2024 by Cluster)
  - **箱型圖繪製**：使用 `plotly.express.box` 繪製箱型圖，將 `Cluster` 作為 x 軸，`Revenue_2024` 作為 y 軸。這樣可以比較不同群集的 2024 年營收分佈。
  - **標題與標籤**：設置圖表標題為 `"Revenue_2024 Distribution by Cluster"`，並且為 x 軸和 y 軸設置適當的標籤。
  - **顯示圖表**：透過 `fig_revenue.show()` 顯示箱型圖，讓使用者可以直觀地了解不同群集間的營收分佈情況。

#### 7.Gemini AI 分析

In [ ]:
prompt= f'我想請你把以下內容分析出哪家公司的財報比較好，包含營收成長、獲利能力、股價表現、財務穩健性等等來進行綜合評估{df.to_string()}並且用正體中文輸出給我'

In [ ]:
from google.colab import userdata
apikey = userdata.get('test')

In [ ]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content(prompt)
print(response.text)

根據提供的資料，要評估哪家公司的財報「最好」需要考量多個面向，並取決於投資人的風險承受能力和投資目標。單純比較數字無法得出絕對的結論，以下提供各公司在營收成長、獲利能力、股價表現（以EPS代表）、財務穩健性等方面的表現分析，供您參考：


**營收成長:**

* **表現優異:** 台泥 (30.41%)、東泥 (26.23%)、聯華 (5.25%)、炎洲 (36.05%)、遠東新 (5.23%)、新纖 (14.19%)、味全 (10.98%)、聯華食 (9.39%)、統一 (15.20%)、達新 (26.11%)、春源 (10.50%)、勤美 (15.35%)、中宇 (16.63%)、佳格 (3.27%)、再生-KY (14.29%)、富林-KY (6.31%)、南僑 (3.87%)、台塑化 (9.09%)、裕隆 (10.10%)、中華 (16.52%)、  華電 (9.64%)、華榮 (20.41%)、大亞 (15.85%)、中鋼 (26.11%)、豐祥-KY (0.89%)、南港 (123.46%)、台橡 (17.31%)、國際中橡 (5.03%)、豐達科 (16.44%)、三星 (7.28%)、士電 (7.40%)、中興電 (14.48%)、亞力 (2.29%)、力山 (29.31%)、華城 (57.34%)、東台 (28.99%)、  信大 (25.15%)、鈞興-KY (28.47%)、儒鴻 (22.28%)、聚陽 (10.01%)、佳和 (9.40%)、宏益 (48.87%)、台富 (5.43%)、集盛 (33.95%)、宜進 (11.77%)、聯發 (26.14%)、宏遠 (4.49%)、台南 (17.09%)、弘裕 (19.36%)、業旺 (55.39%)、東和鋼鐵 (1.57%)、燁興 (38.12%)、第一銅 (21.54%)、中鴻 (20.17%)、春雨 (5.80%)、中鋼構 (0.13%)、官田鋼 (5.29%)、美亞 (28.50%)、燁輝 (2.31%)、志聯 (20.91%)、千興 (55.83%)、大成鋼 (-13.13%)、威致 (1.21%)、盛餘 (11.58%)、彰源 (-4.63%)、新光鋼 (-12.11%)、新鋼 (-0.14%)、允強 (-11.38%)、海光 (4.32%)、運錩 (-11.93%)、長榮鋼 (

In [ ]:
from io import StringIO
df_all = pd.read_csv(StringIO(response.text))
df_all

,根據提供的資料，要評估哪家公司的財報「最好」需要考量多個面向，並取決於投資人的風險承受能力和投資目標。單純比較數字無法得出絕對的結論，以下提供各公司在營收成長、獲利能力、股價表現（以EPS代表）、財務穩健性等方面的表現分析，供您參考：
0,**營收成長:**
1,* **表現優異:** 台泥 (30.41%)、東泥 (26.23%)、聯華 (5.25%)...
2,* **表現不佳:** 亞泥 (-7.97%)、幸福 (-3.66%)、信大 (-23.64...
3,**獲利能力 (以稅後淨利和營業利益來看):**
4,這部分需要更詳細的資料才能精準判斷，因為提供的資料中「非營業收入」和「非營業支出」項目影響淨...
5,* **稅後淨利大幅成長:** 嘉泥(282.37倍) 此數據需謹慎看待，因基期過低，可能...
6,**股價表現 (以EPS代表):**
7,* **表現優異:** 嘉泥(282.37倍) 同上，此數據需謹慎看待。 中興電(248....
8,**財務穩健性 (速動比率和流動比率):**
9,速動比率和流動比率越高，代表公司償債能力越好，財務越穩健。 各公司比率差異大，需個別分析：


#### 7. 回傳google sheet 紀錄

In [ ]:
# 選擇要更新的工作表（選擇第一個工作表）
worksheet2 = gsheets.get_worksheet(2)  # 如果是其他工作表，可以用 get_worksheet(index)

# 將 DataFrame 寫入 Google Sheets
set_with_dataframe(worksheet2, df_all)